In [1]:
import os
os.chdir("../../")
import numpy as np
from backtesting import Backtest
from backtesting.lib import Strategy

from backtesting.test import EURUSD
from strategies.ichimoku_v1 import IchimokuCloud

/Users/pimpmasterguapo/Documents/github/personal_projects/ai_trading_app/.venv/lib/python3.11/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/Users/pimpmasterguapo/Documents/github/personal_projects/ai_trading_app/.venv/lib/python3.11/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),


In [2]:
class IchimokuCloudBacktest(Strategy):
    def init(self):
        self.position_open = False
        self.data.df.reset_index(inplace=True, names="Datetime")

    def calculate_risk_reward_ratio(
        self, entry_price, stop_loss_price, take_profit_price
    ):
        risk_reward_ratio = (take_profit_price - entry_price) / (
            entry_price - stop_loss_price
        )
        return risk_reward_ratio

    def calculate_take_profit(self, entry_price, stop_loss_price, risk_reward_ratio, is_buy=True):
        direction_factor = 1 if is_buy else -1
        take_profit_price = entry_price + direction_factor * risk_reward_ratio * abs(entry_price - stop_loss_price)
        return take_profit_price

    def calculate_pip_amount(self, entry_price, exit_price, pip_factor=0.0001):
        pip_amount = (exit_price - entry_price) / pip_factor
        return pip_amount
    
    def calculate_new_price(self, original_price, pip_value, pip_factor=0.0001, add_pips=True):
        direction = 1 if add_pips else -1
        new_price = original_price + direction * pip_value * pip_factor
        return new_price


    def next(self):
        if self.position:
            pass
        else:
            ichimoku = IchimokuCloud(self.data.df)
            ichimoku_df = ichimoku.get_ichimoku_df()
            price_cloud_cross_signal = ichimoku.price_senkou_span_cross(ichimoku_df)
            kijun_sen_direction_signal = ichimoku.kijun_sen_direction(ichimoku_df)

            mandatory_signals = np.sum(
                [price_cloud_cross_signal, kijun_sen_direction_signal]
            )
            if mandatory_signals == 2 or mandatory_signals == -2:
                (
                    senkou_span_cloud_signal,
                    senkou_span_cloud_grade,
                ) = ichimoku.senkou_span_cloud_color(ichimoku_df)
                (
                    price_kijun_cross_signal,
                    price_kijun_cross_grade,
                ) = ichimoku.price_kijun_sen_cross(ichimoku_df)
                (
                    tenken_kijun_cross_signal,
                    tenken_kijun_cross_grade,
                ) = ichimoku.tenkan_sen_kijun_sen_cross(ichimoku_df)
                (
                    price_chikou_cross_signal,
                    price_chikou_cross_grade,
                ) = ichimoku.price_chikou_span_cross(ichimoku_df)
                confirmation_grades = np.sum(
                    [
                        senkou_span_cloud_grade,
                        price_kijun_cross_grade,
                        tenken_kijun_cross_grade,
                        price_chikou_cross_grade,
                    ]
                )
                if confirmation_grades >= 10:
                    confirmation_signals = np.sum(
                        [
                            senkou_span_cloud_signal,
                            price_kijun_cross_signal,
                            tenken_kijun_cross_signal,
                            price_chikou_cross_signal,
                        ]
                    )
                    if mandatory_signals == 2 and confirmation_signals >= 2:
                        last_close_index = ichimoku_df["Close"].last_valid_index()
                        if last_close_index != None:
                            close = self.data.df.iloc[-1]["Close"]
                            senkou_span_a = ichimoku_df.iloc[last_close_index][
                                "senkou_span_a"
                            ]
                            senkou_span_b = ichimoku_df.iloc[last_close_index][
                                "senkou_span_b"
                            ]
                            senkou_line = np.max([senkou_span_a, senkou_span_b])
                            senkou_stop_loss = senkou_line - (senkou_line * 0.05)
                            pip_stop_loss = self.calculate_new_price(close, 200, add_pips=False)
                            stop_loss = np.max([senkou_stop_loss, pip_stop_loss])
                            take_profit = self.calculate_take_profit(close, stop_loss, 1.5, True)

                            print(
                                f"""BUY -> Close: {close}
                                TP: {take_profit} TPPIPS: {self.calculate_pip_amount(close, take_profit)}
                                SL: {stop_loss} SLPIPS: {self.calculate_pip_amount(close, stop_loss)}
                                RRR: {self.calculate_risk_reward_ratio(close,stop_loss,take_profit)}""".strip()
                            )

                            self.buy(sl=stop_loss, tp=take_profit)
                            self.position_open = True

                    elif mandatory_signals == -2 and confirmation_signals <= -2:
                        last_close_index = ichimoku_df["Close"].last_valid_index()
                        if last_close_index != None:
                            close = self.data.df.iloc[-1]["Close"]
                            senkou_span_a = ichimoku_df.iloc[last_close_index][
                                "senkou_span_a"
                            ]
                            senkou_span_b = ichimoku_df.iloc[last_close_index][
                                "senkou_span_b"
                            ]
                            senkou_line = np.min([senkou_span_a, senkou_span_b])
                            senkou_stop_loss = senkou_line * 1.05
                            pip_stop_loss = self.calculate_new_price(close, 200)
                            stop_loss = np.min([senkou_stop_loss, pip_stop_loss])
                            take_profit = self.calculate_take_profit(close, stop_loss, 1.5, False)

                            print(
                                f"""SELL -> Close: {close}
                                TP: {take_profit} TPPIPS: {self.calculate_pip_amount(take_profit, close)}
                                SL: {stop_loss} SLPIPS: {self.calculate_pip_amount(stop_loss, close)}
                                RRR: {self.calculate_risk_reward_ratio(close,stop_loss,take_profit)}""".strip()
                            )

                            self.sell(sl=stop_loss, tp=take_profit)
                            self.position_open = True

In [3]:
bt = Backtest(EURUSD, IchimokuCloudBacktest,
              cash=10000, commission=.002,
              exclusive_orders=True)

output = bt.run()
bt.plot()

SELL -> Close: 1.0701
                                TP: 1.0401 TPPIPS: 300.0000000000002
                                SL: 1.0901 SLPIPS: -200.00000000000017
                                RRR: 1.5
BUY -> Close: 1.08405
                                TP: 1.11405 TPPIPS: 300.0000000000002
                                SL: 1.06405 SLPIPS: -200.00000000000017
                                RRR: 1.5
BUY -> Close: 1.11466
                                TP: 1.14466 TPPIPS: 300.0000000000002
                                SL: 1.09466 SLPIPS: -200.00000000000017
                                RRR: 1.5
BUY -> Close: 1.14396
                                TP: 1.1739600000000001 TPPIPS: 300.0000000000002
                                SL: 1.12396 SLPIPS: -200.00000000000017
                                RRR: 1.5
BUY -> Close: 1.17399
                                TP: 1.2039900000000001 TPPIPS: 300.0000000000002
                                SL: 1.15399 SLPIPS: -200.00000000000

/Users/pimpmasterguapo/Documents/github/personal_projects/ai_trading_app/.venv/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/pimpmasterguapo/Documents/github/personal_projects/ai_trading_app/.venv/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/pimpmasterguapo/Documents/github/personal_projects/ai_trading_app/.venv/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/Users/pimpmasterguapo/Documents/github/personal_projects/ai_trading_app/.venv/lib/python3.11/site-packages/backtesting/_plotting.py:659

GridPlot(id='p1305', ...)

In [4]:
output

Start                     2017-04-19 09:00:00
End                       2018-02-07 15:00:00
Duration                    294 days 06:00:00
Exposure Time [%]                       98.72
Equity Final [$]                 10671.225314
Equity Peak [$]                  10820.178664
Return [%]                           6.712253
Buy & Hold Return [%]               14.628937
Return (Ann.) [%]                    6.739877
Volatility (Ann.) [%]                 7.26024
Sharpe Ratio                         0.928327
Sortino Ratio                        1.527288
Calmar Ratio                         0.993208
Max. Drawdown [%]                   -6.785965
Avg. Drawdown [%]                   -0.545397
Max. Drawdown Duration      149 days 06:00:00
Avg. Drawdown Duration        6 days 02:00:00
# Trades                                   13
Win Rate [%]                        53.846154
Best Trade [%]                       2.564168
Worst Trade [%]                     -2.071223
Avg. Trade [%]                    

In [5]:
output["_trades"]

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-9363,52,60,1.067980,1.09010,-207.111807,-0.020712,52,60,8
1,9015,65,485,1.086198,1.11405,251.085239,0.025642,65,485,420
2,8992,486,1423,1.116909,1.14466,249.533755,0.024846,486,1423,937
3,8980,1424,1690,1.146268,1.17396,248.674519,0.024158,1424,1690,266
4,8961,1697,2254,1.176328,1.20399,247.879540,0.023516,1697,2254,557
5,8938,2255,2305,1.207109,1.18467,-200.563357,-0.018589,2255,2305,50
6,-8946,2306,2441,1.183708,1.20605,-199.872963,-0.018875,2306,2441,135
7,8589,2442,2536,1.209514,1.18715,-192.086114,-0.018490,2442,2536,94
8,-8604,2537,3296,1.185205,1.15764,237.167883,0.023257,2537,3296,759
9,-9028,3297,3585,1.155784,1.17806,-201.109534,-0.019274,3297,3585,288
